# Fetch Leaderboard Data

Goals:
* Request position data from The Graph
* Rank users on their profit and loss within a specified time period

In [1]:
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal


In [2]:
# set up gql
# FUTURES_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-main' # mainnet
FUTURES_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-kovan-main' # testnet


In [3]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)

def clean_df(df, decimal_cols):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=FUTURES_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        return result



In [4]:
# queries
userMarginAccountsBlock = gql("""
query marginAccounts($block_number: Int!) {
  futuresMarginAccounts(
    block: {
      number: $block_number
    }
  ) {
    id
    timestamp
    account
    market
    asset
    margin
    deposits
    withdrawals
  }  
}
""")

allMarginAccountsBlock = gql("""
query marginAccounts($block_number: Int!) {
  futuresMarginAccounts(
    block: {
      number: $block_number
    }
  ) {
    id
    timestamp
    account
    market
    asset
    margin
    deposits
    withdrawals
  }  
}
""")


### Query user account by block

In [5]:
## only relevant for testnet
## blocks
deposit_block = 3803686
open_block = 3803886
close_block = 3804060

blocks = [
    deposit_block,
    open_block,
    close_block
]



In [6]:
df_user = pd.DataFrame()
for block in blocks:
    params = {
        'account': '0x268671dd4Dc2417b56EDDca2eDb6b2D07e8Ffb13',
        'block_number': block
    }

    decimal_cols = [
        'margin',
        'deposits',
        'withdrawals'
    ]

    margin_response = await run_query(userMarginAccountsBlock, params)
    df_margin = pd.DataFrame(margin_response['futuresMarginAccounts'])
    df_margin = clean_df(df_margin, decimal_cols)
    df_margin['block'] = block
    df_user = pd.concat([df_user, df_margin])

df_user.head()

,id,timestamp,account,market,asset,margin,deposits,withdrawals,block
0,0x0a28f7f8ba0659361cee7e072277ba6eac518d0e-0x8...,1650270483,0x0a28f7f8ba0659361cee7e072277ba6eac518d0e,0x8e0df45f66e620f85df1d0490cd2b19e57a4232a,0x4d415449430000000000000000000000000000000000...,0,500,0,3803686
1,0x0a28f7f8ba0659361cee7e072277ba6eac518d0e-0x9...,1649433724,0x0a28f7f8ba0659361cee7e072277ba6eac518d0e,0x929d8ec9a885cdcfdf28ea31b4a356532757de5e,0x57544900000000000000000000000000000000000000...,198.999999999999999881,200,0,3803686
2,0x15adbea538f541271da5e4436e41285e386e3336-0x6...,1653516686,0x15adbea538f541271da5e4436e41285e386e3336,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,956.262818026642189776,1000,0,3803686
3,0x17e0703e5776fc7c96e17a37a8b2cf979abe546d-0x6...,1653959337,0x17e0703e5776fc7c96e17a37a8b2cf979abe546d,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,0,333.333333333333333333,331.362557314407583735,3803686
4,0x1a5affd2d721729b708e1b9e9925bcc3e01e788d-0x6...,1649826442,0x1a5affd2d721729b708e1b9e9925bcc3e01e788d,0x698e403aac625345c6e5fc2d0042274350bedf78,0x73455448000000000000000000000000000000000000...,50,219.805469754980742952,169.610939509961485904,3803686


## Query all users between two blocks

In [7]:
lb_blocks = [
    2557167,
    3860759
]

lb_results = {}
for block in lb_blocks:
    params = {
        'block_number': block
    }

    decimal_cols = [
        'margin',
        'deposits',
        'withdrawals'
    ]

    margin_response = await run_query(allMarginAccountsBlock, params)
    df_margin = pd.DataFrame(margin_response['futuresMarginAccounts'])
    df_margin = clean_df(df_margin, decimal_cols)

    # summarize the data by account
    df_margin = df_margin.groupby('account')[[
        'margin',
        'deposits',
        'withdrawals'
    ]].sum().reset_index()

    lb_results[block] = df_margin


In [8]:
## calculate the leaderboard
# get the start and end data
start_df = lb_results[lb_blocks[0]]
end_df = lb_results[lb_blocks[1]]

# merge together
df_lb = start_df.merge(
    end_df,
    on='account',
    suffixes=('_start', '_end')
)

# calculated fields
df_lb['margin_change'] = df_lb['margin_end'] - df_lb['margin_start']
df_lb['deposits_change'] = df_lb['deposits_end'] - df_lb['deposits_start']
df_lb['withdrawals_change'] = df_lb['withdrawals_end'] - df_lb['withdrawals_start']


df_lb['pnl'] = df_lb['margin_change'] - df_lb['deposits_change'] + df_lb['withdrawals_change']
df_lb['pnl_pct'] = df_lb['pnl'] / (df_lb['margin_start'] + df_lb['deposits_change']).apply(lambda x: max(500, x))


In [9]:
df_lb.sort_values('pnl_pct', ascending=False)[[
    'account',
    'pnl',
    'pnl_pct'
]]

,account,pnl,pnl_pct
24,0xa366905cd45758801d4acebcc046ca4651c4ff40,952.609999563682669975,0.1688353441654155183664561602
12,0x64af258f8522bb9dd2c2b648b9cbbcdbe986b0bf,296.394569144606519493,0.05550779153365982125446657731
25,0xb0cffe0260bf4ea7b59915fbea17273a8b9209f6,2.693074380195831485,0.00538614876039166297
11,0x540a5a8ea7f027bff4f1a84815499a080cac02a1,0,0
23,0xa19623376fcc9087229f67a82089b5363fe8ca27,0E-18,0E-18
19,0x8ff5b4403878fd09abf4b5c3b8da237bcdfd0ff3,0E-18,0E-18
18,0x8f2b35e442ea87df1d2cec16a2565e93c578c66a,0E-18,0E-18
17,0x79a29ae677b40312b2952d18e227c282844a30fd,0E-18,0
15,0x72784ca2f96bd74a8036fbabecaa8ab6b83c9605,0E-18,0
14,0x706d961ab69d54a0fcbaa13e77842279a5724139,0E-18,0E-18
